In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir

In [2]:
listdir()

['.ipynb_checkpoints', 'MI', 'C', 'Classification.ipynb', '.DS_Store']

In [9]:
c_data_path = ['./C/' + i for i in listdir('./C')]
mi_data_path =['./MI/' + i for i in listdir('./MI')]


In [61]:
def find_name(s:str):
    s = list(s)
    ans = ''
    for i in s[4:]:
        if not i.islower() and i!='_' and i!='.':
            ans+= i
    return ans
def data_labels_by_name(name:str, l:list):
    ans = ['','']
    for i in l:
        if i.count(name):
            if i.count('data'):
                ans[0]=i
            if i.count('labels'):
                ans[1]=i
    return ans

In [60]:
names = set([find_name(i) for i in c_data_path])
patients = []
for name in names:
    patients.append((data_labels_by_name(name,c_data_path)+data_labels_by_name(name,mi_data_path)))

Choosing random participant

In [62]:
import random
data = random.choice(patients)
data

['./C/NIKITA_data.npy',
 './C/NIKITA_labels.npy',
 './MI/NIKITA_data.npy',
 './MI/NIKITA_labels.npy']

In [72]:
data_c, labels_c, data_mi, labels_mi = [np.load(i) for i in data]
labels_mi *= 2

## Classification

In [144]:
from sklearn import ensemble
rf_clf = ensemble.RandomForestClassifier()

In [79]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()

In [167]:
data = np.concatenate((data_c,data_mi),axis=0)

In [168]:
data = data - data.mean(axis = 2).reshape(720,24,1)
labels = np.concatenate((labels_c,labels_mi),axis=0)

In [169]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
data_open = data.reshape(720,-1)

In [170]:
from sklearn.model_selection import train_test_split
x_tr, x_test, y_train, y_test = train_test_split(data_open,labels)

In [171]:
lda.fit(x_tr, y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [334]:
lda.score(x_test,y_test)

0.45

In [173]:
rf_clf.fit(x_tr,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [174]:
rf_clf.score(x_test,y_test)

0.5888888888888889

In [348]:
def test(classifier, n_samples, data, labels):
    pred = []
    samples_i = np.random.choice(range(len(data)), size=n_samples)
    samples = data[samples_i]
    label_true = labels[samples_i]
    label_predict = []
    true_freq =  np.argmax(np.bincount(label_true))
    for i in range(n_samples):
        label_predict.append(classifier.predict(samples[i].reshape(1,-1))[0])
    pred_freq = np.argmax(np.bincount(label_predict))
    return true_freq==pred_freq

In [353]:
log = []
for i in range(30):
    log.append(test(rf_clf,3, x_test,y_test))
print(sum(log)/len(log)) #процент правильно предсказанных лейблов из {0,1,2}

0.6333333333333333


In [354]:
def group_classifier(classifier, samples):
    pred = []
    label_predict = []
    for i in range(len(samples)):
        label_predict.append(classifier.predict(samples[i].reshape(1,-1))[0])
    pred_freq = np.argmax(np.bincount(label_predict))
    return pred_freq